In [1]:
import keras
import wandb
from wandb.keras import WandbCallback
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.applications.resnet50 import ResNet50, decode_predictions, preprocess_input
from keras.applications.xception import Xception
import DatasetParser
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model

Using TensorFlow backend.
wandb: WARNING Keras version 2.3.1 is not fully supported. Required keras >= 2.4.0


In [2]:
(x_train, y_train_raw), (x_test, y_test_raw), class_names = DatasetParser.load_data("Xception")

299
Entrenamiento: 204
Test: 80


In [3]:
# One hot encode ouput
y_train = keras.utils.to_categorical(y_train_raw)
y_test = keras.utils.to_categorical(y_test_raw)

In [4]:
Xception_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(299, 299, 3),
    include_top=True)

In [5]:
from keras.preprocessing import image
import numpy as np
img = image.load_img('elephant.jpg', target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = Xception_model.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n03775546', 'mixing_bowl', 0.90169585), ('n02112350', 'keeshond', 0.09713012), ('n03942813', 'ping-pong_ball', 0.0011666301)]


In [6]:
# We should preprocess the images the same way resnet images were preprocessed
x_train_preprocessed = preprocess_input(x_train)
x_test_preprocessed = preprocess_input(x_test)

In [7]:
# Build a new model that is ResNet50 minus the very last layer
last_layer = Xception_model.get_layer("avg_pool")

Xception_layers = keras.Model(inputs=Xception_model.inputs, outputs=last_layer.output)
Xception_layers.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [8]:
# We use our resnet to "predict" but because we have removed the top layer, 
# this outputs the activations of the second to last layer on our dataset

x_train_features = Xception_layers.predict(x_train_preprocessed)

In [9]:
x_test_features = Xception_layers.predict(x_test_preprocessed)

In [10]:
# We can directly stich the models together

Fine_Tuning_model=Sequential()
Fine_Tuning_model.add(Xception_layers)
Fine_Tuning_model.add(Dense(2, activation="sigmoid"))

Fine_Tuning_model.layers[0].trainable=False

Fine_Tuning_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

Fine_Tuning_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 2048)              20861480  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098      
Total params: 20,865,578
Trainable params: 4,098
Non-trainable params: 20,861,480
_________________________________________________________________


In [11]:
wandb.init(project="tf-covid-19")
Fine_Tuning_model.fit(x_train_preprocessed, y_train, epochs=15, validation_data=(x_test_preprocessed, y_test), callbacks=[WandbCallback()])

wandb: Currently logged in as: scueroinc (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Train on 204 samples, validate on 80 samples
Epoch 1/15
204/204 [==============================] - 7s 34ms/step - loss: 0.6448 - accuracy: 0.6814 - val_loss: 0.3450 - val_accuracy: 0.8875
Epoch 2/15
204/204 [==============================] - 5s 26ms/step - loss: 0.5298 - accuracy: 0.8971 - val_loss: 1.1786 - val_accuracy: 0.6500
Epoch 3/15
204/204 [==============================] - 5s 26ms/step - loss: 0.4221 - accuracy: 0.9118 - val_loss: 0.5043 - val_accuracy: 0.8750
Epoch 4/15
204/204 [==============================] - 5s 26ms/step - loss: 0.3487 - accuracy: 0.9265 - val_loss: 0.5011 - val_accuracy: 0.8750
Epoch 5/15
204/204 [==============================] - 5s 26ms/step - loss: 0.2508 - accuracy: 0.9657 - val_loss: 1.0387 - val_accuracy: 0.7750
Epoch 6/15
204/204 [==============================] - 5s 26ms/step - loss: 0.2216 - accuracy: 0.9608 - val_loss: 1.3555 - val_accuracy: 0.7500
Epoch 7/15
204/204 [==============================] - 5s 26ms/step - loss: 0.1848 - accuracy: 0.9

In [12]:
# We can allow some of the resnet layers to change as we train.  
# Typically you would want to lower the learning rate in conjunction with this.

Fine_Tuning_model.layers[0].trainable = True

# We let the last 3 blocks train
for layer in Fine_Tuning_model.layers[0].layers[:-11]:
    layer.trainable = False
for layer in Fine_Tuning_model.layers[0].layers[-11:]:
    layer.trainable = True
    
Fine_Tuning_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
Fine_Tuning_model.fit(x_train_preprocessed, y_train, epochs=15, validation_data=(x_test_preprocessed, y_test), callbacks=[WandbCallback()])

Train on 204 samples, validate on 80 samples
Epoch 1/15
204/204 [==============================] - 9s 45ms/step - loss: 0.0386 - accuracy: 0.9853 - val_loss: 0.5402 - val_accuracy: 0.7875
Epoch 2/15
204/204 [==============================] - 6s 28ms/step - loss: 0.0611 - accuracy: 0.9804 - val_loss: 0.3406 - val_accuracy: 0.9250
Epoch 3/15
204/204 [==============================] - 6s 28ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 1.5787 - val_accuracy: 0.7500
Epoch 4/15
204/204 [==============================] - 6s 28ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 2.2925 - val_accuracy: 0.7000
Epoch 5/15
204/204 [==============================] - 6s 28ms/step - loss: 5.6332e-04 - accuracy: 1.0000 - val_loss: 2.0563 - val_accuracy: 0.7375
Epoch 6/15
204/204 [==============================] - 6s 28ms/step - loss: 6.7703e-04 - accuracy: 1.0000 - val_loss: 1.5626 - val_accuracy: 0.8000
Epoch 7/15
204/204 [==============================] - 6s 28ms/step - loss: 1.0356e-04 - a